<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/master/lecture_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**
# Class 5: Natural language processing (NLP)

# LECTURE

## Setup

We will be using [spaCy](https://spacy.io/) for NLP. First, we need to download the model:

In [1]:
!python -m spacy download en_core_web_md

Could not build wheels for en-core-web-md, since package 'wheel' is not installed.
Could not build wheels for spacy, since package 'wheel' is not installed.
Could not build wheels for preshed, since package 'wheel' is not installed.
Could not build wheels for murmurhash, since package 'wheel' is not installed.
Could not build wheels for tqdm, since package 'wheel' is not installed.
Could not build wheels for plac, since package 'wheel' is not installed.
Could not build wheels for numpy, since package 'wheel' is not installed.
Could not build wheels for wasabi, since package 'wheel' is not installed.
Could not build wheels for blis, since package 'wheel' is not installed.
Could not build wheels for requests, since package 'wheel' is not installed.
Could not build wheels for setuptools, since package 'wheel' is not installed.
Could not build wheels for catalogue, since package 'wheel' is not installed.
Could not build wheels for thinc, since package 'wheel' is not installed.
Could not bu

In [2]:
import spacy

nlp = spacy.load("en_core_web_md")

## Concepts

- Model
- Training
- Document
- [Features](https://spacy.io/usage/spacy-101#features)
  - Tokens
  - Parts of speech
  - Lemmas
  - Entities
- Vectors

Example from [DataCamp](https://campus.datacamp.com/courses/advanced-nlp-with-spacy/finding-words-phrases-names-and-concepts?ex=4).

In [3]:
doc = nlp("In 1990, more than 60% of people in East Asia were in extreme poverty. Now less than 4% are.")

In [4]:
from spacy import displacy

displacy.render(doc)

## Entities

In [5]:
displacy.render(doc, style='ent')

## Sentiment analysis

We are going to use a different library, [TextBlob](https://textblob.readthedocs.io/). Works similarly to spaCy, though.

In [6]:
from textblob import TextBlob

TextBlob("I love this class!").sentiment

Sentiment(polarity=0.625, subjectivity=0.6)

In [8]:
TextBlob("I hate pandas").sentiment

Sentiment(polarity=-0.8, subjectivity=0.9)

Let's get some real data to work with: [Public comments from the proposed "Traveling by Air with Service Animals" rule](https://beta.regulations.gov/document/DOT-OST-2018-0068-12959/comment).

In [33]:
import pandas as pd

pd.options.display.max_colwidth = None

df = pd.read_csv('service_animals.csv')
df

,displayProperties,cfrPart,country,subject,ombApproval,frDocNum,withdrawn,agencyId,allowLateComments,fileFormats,...,organization,duplicateComments,trackingNbr,commentOn,field2,submitterRepAddress,openForComment,effectiveDate,id,type
0,NaN,NaN,United States,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f5t-i65a,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-14971,documentdetails
1,NaN,NaN,United States,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f5d-81f2,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-14917,documentdetails
2,NaN,NaN,United States,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f5e-cn4v,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-14918,documentdetails
3,NaN,NaN,United States,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f5e-pt6q,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-14919,documentdetails
4,NaN,NaN,NaN,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f5f-iuo7,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-14920,documentdetails
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,NaN,NaN,NaN,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f6i-cj4c,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-15072,documentdetails
496,NaN,NaN,NaN,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f6j-kw6m,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-15074,documentdetails
497,NaN,NaN,NaN,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f6j-hewt,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-15075,documentdetails
498,NaN,NaN,NaN,NaN,NaN,NaN,False,DOT,False,NaN,...,NaN,0,1k4-9f6k-cjht,090000648432b7e1,NaN,NaN,False,NaN,DOT-OST-2018-0068-15078,documentdetails


In [34]:
df.content

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Cleaning up text

First, stripping HTML tags using [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

In [35]:
from bs4 import BeautifulSoup


html_str = '''
This is some text<br/>
with some <em>HTML</em> in it.
'''
soup = BeautifulSoup(html_str)

print(soup.get_text())

This is some text
with some HTML in it.



In [39]:
def strip_html(text):
    soup = BeautifulSoup(text)
    return soup.get_text()


df['clean_content'] = df.content.fillna('').apply(strip_html)

changed_content = df[df.content != df.clean_content]
changed_content[['content', 'clean_content']]

,content,clean_content
0,"My wife was depressed on a daily basis. She cried all the time. Once we got our little puppy, my wife feels she has a purpose and has not cried since. She needs to have him with her when travelling or she is very stressed and depressed. He has made such a difference. Please don&#39;t take away the ability for our dog to travel with her.","My wife was depressed on a daily basis. She cried all the time. Once we got our little puppy, my wife feels she has a purpose and has not cried since. She needs to have him with her when travelling or she is very stressed and depressed. He has made such a difference. Please don't take away the ability for our dog to travel with her."
1,"Docket number (DOT-OST-2018-0068) and the Regulatory ID number (RIN No. 2105-AE63)<br/>I am very much in favor of amended regulations that provide strict definitions for what type of animals might be permitted to accompany flyers in the cabin of commercial aircraft. As I have read the pending rules, only a properly trained service dog would fulfill the criteria to accompany a passenger with special needs in the cabin. The wide and random acceptance of non-professional trained emotional support animals in the cabin of aircraft would cease. This is essential for the health and safety of the passengers at large as well as the hard working crew. I hope to see this change made very soon. Thank you.","Docket number (DOT-OST-2018-0068) and the Regulatory ID number (RIN No. 2105-AE63)I am very much in favor of amended regulations that provide strict definitions for what type of animals might be permitted to accompany flyers in the cabin of commercial aircraft. As I have read the pending rules, only a properly trained service dog would fulfill the criteria to accompany a passenger with special needs in the cabin. The wide and random acceptance of non-professional trained emotional support animals in the cabin of aircraft would cease. This is essential for the health and safety of the passengers at large as well as the hard working crew. I hope to see this change made very soon. Thank you."
3,"RE: DOCKET NUMBER DOT-OST-2018-0068<br/><br/>I am a psychotherapist who has worked with many people with PTSD and Severe panic attacks. I have spoken with men whose PTSD, from serving in the military, has been so severe that they cannot go anywhere without their emotional support animals. I have also treated people who have such bad panic attacks that they become paralyzed or sometimes even faint.<br/><br/>I implore you to not take away the privilege of these people having their ESAs on planes, to provide necessary comfort. Just because you cannot see a disorder, doesn&#39;t mean it doesn&#39;t exist. I liken it to a domestic violence survivor who was emotionally abused. His or her pain can be just as great as one who was physically abused.<br/><br/>You will be doing a tremendous disservice to people who legitimately need their ESAs, if you don&#39;t let these people travel<br/>with them. How can passengers prove that the service their ESAs serve is to help them feel safe? This is not an observerable task.<br/><br/>Clearly people have abused this policy and that couldn&#39;t be more wrong. However, there must be a better solution than what <br/>the DOT is proposing. Some airlines are having passengers sign waivers that their pets are well-trained. Can&#39;t that be enough? Also, can&#39;t the proposal just consider allowing people with just dogs and cats as ESAs?<br/><br/>Please consider what I have said. If this rule is instituted, many people&#39;s lives will be severely negatively affected.<br/><br/>Thank you.","RE: DOCKET NUMBER DOT-OST-2018-0068I am a psychotherapist who has worked with many people with PTSD and Severe panic attacks. I have spoken with men whose PTSD, from serving in the military, has been so severe that they cannot go anywhere without their emotional support animals. I have also treated people who have such bad panic attacks that th

In [45]:
def sentiment(row):
    blob = TextBlob(row.clean_content)
    return blob.sentiment.polarity

df['sentiment'] = df.apply(sentiment, axis=1)
df[['clean_content', 'sentiment']].sort_values('sentiment', ascending=False)

,clean_content,sentiment
94,"I am in favor of the proposed modification. Under the umbrella of Service Animal, all should be trained. This protects the traveling public.This is best for the animals as well. Unlike a trained Service Animal an untrained ESA may be stressed in an unfamiliar environment.This is a win for all Travelers.",0.900000
46,"I dont think size limits should be imposed. that would exclude many great dane service dog partnerships from traveling. Great Danes are used for mobility and stability for those with balance disorders, MS, Parkinson's, etc.",0.700000
41,Please help keep our flights safe by removing untrained ESAs. Thank you!,0.625000
311,"Please require service animals to meet more requirements, be registered as service animals with Carriers. There are many passengers who abuse the SVAN and ESAN programs.",0.500000
159,"""The Seeing Eye"" has submitted comments regarding these proposed regulations. Each and everyone of their recommendations need to be fulfilled. I support their position 100%. Please don't enact these regulations until ""The Seeing Eye"" recommendations are adhered to. Sincerely, Marijean Hansen.",0.500000
...,...,...
107,We deal with passengers everyday with fake letters purchased from a website like esa.com The animals are out of control. There should b a verification process proof of training.,-0.350000
449,Dogs and cats can't even catch corona! This rule is ridiculous and will negatively affect A LOT of people!,-0.354167
219,"yes please eliminate fake service animals from flights. as a person who suffers from a severe allergy to cats, there is no reason to have one onboard a plane. take the train . ride a bus ,walk.the only animals allowed in air cabins should be seeing eye dogs .......period....",-0.366667
411,The emotional support animals need to STOP!!!! It's out of control as they are fake. Please take a stand against these people that are abusing the system!!,-0.390625


## Vectors

In short, they represent the "meaning" of a word as a bunch of numbers, which corresponds to a point in space. [More info.](https://www.youtube.com/watch?v=LSS_bos_TPI&list=PLRqwX-V7Uu6aQ0oh9nH8c6U1j9gCg-GdF&index=2&t=336s)

In [46]:
nlp('apple').vector

array([-3.6391e-01,  4.3771e-01, -2.0447e-01, -2.2889e-01, -1.4227e-01,
        2.7396e-01, -1.1435e-02, -1.8578e-01,  3.7361e-01,  7.5339e-01,
       -3.0591e-01,  2.3741e-02, -7.7876e-01, -1.3802e-01,  6.6992e-02,
       -6.4303e-02, -4.0024e-01,  1.5309e+00, -1.3897e-02, -1.5657e-01,
        2.5366e-01,  2.1610e-01, -3.2720e-01,  3.4974e-01, -6.4845e-02,
       -2.9501e-01, -6.3923e-01, -6.2017e-02,  2.4559e-01, -6.9334e-02,
       -3.9967e-01,  3.0925e-02,  4.9033e-01,  6.7524e-01,  1.9481e-01,
        5.1488e-01, -3.1149e-01, -7.9939e-02, -6.2096e-01, -5.3277e-03,
       -1.1264e-01,  8.3528e-02, -7.6947e-03, -1.0788e-01,  1.6628e-01,
        4.2273e-01, -1.9009e-01, -2.9035e-01,  4.5630e-02,  1.0120e-01,
       -4.0855e-01, -3.5000e-01, -3.6175e-01, -4.1396e-01,  5.9485e-01,
       -1.1524e+00,  3.2424e-02,  3.4364e-01, -1.9209e-01,  4.3255e-02,
        4.9227e-02, -5.4258e-01,  9.1275e-01,  2.9576e-01,  2.3658e-02,
       -6.8737e-01, -1.9503e-01, -1.1059e-01, -2.2567e-01,  2.41

In [47]:
apple = nlp('apple')
pear = nlp('pear')
apple.similarity(pear)

0.5968762340877272

In [48]:
truck = nlp('truck')
apple.similarity(truck)

0.18432754056976158

In [51]:
df['nlp'] = df.clean_content.apply(lambda x: nlp(x))

In [52]:
import itertools
import editdistance

dupes = 0

# initialize column
df['dupe'] = False

for idxA, idxB in itertools.combinations(df.index, 2):
    nlpA = df.nlp[idxA]
    nlpB = df.nlp[idxB]

    similarity = nlpA.similarity(nlpB)
    if similarity > 0.99:
#         print(similarity)
#         print(nlpA)
#         print("---------")
#         print(nlpB)
#         break
        dupes += 1
        df.at[idxA, 'dupe'] = True
        df.at[idxB, 'dupe'] = True
        
dupes

KeyboardInterrupt: 

In [ ]:
import editdistance
import itertools

dupes = 0

# initialize column
df['dupe'] = False

for idxA, idxB in itertools.combinations(df.index, 2):
    contentA = df.clean_content[idxA]
    contentB = df.clean_content[idxB]

    # compute the edit distance ratio
    distance = editdistance.eval(contentA, contentB)
    max_chars = max(len(contentA), len(contentA))

    # prevent divide-by-zero
    if max_chars == 0:
        ratio = 0.0
    else:
        ratio = float(distance) / max_chars

    if ratio < 0.2:
        dupes += 1
        df.at[idxA, 'dupe'] = True
        df.at[idxB, 'dupe'] = True
        
dupes

In [ ]:
df[['content', 'dupe']]